In [1]:
from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet, SWINUNETR
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity
from src.model.blocks import *
import torch.nn as nn
from src.utils.metrics import *

ModuleNotFoundError: No module named 'src'

In [ ]:
example2 = torch.rand(size=[1, 1, 64, 64, 64]).cuda()
mask = torch.rand(size=[1, 3, 128, 128, 128]).cuda()
loss_fn = monai.losses.DiceCELoss()

In [ ]:
# modes = normal, skip, autoencoder, half_half, no_vit
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=32,
                 big_channel=32,
                 patch_size=8,
                 embed_dim=512,
                 mode="normal",
                 old_embedder=True).cuda()
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-4)

# model = UNet.cuda()

In [ ]:
%%time
out = model(example2)

l = loss_fn(out, mask)
l.backward()
optimizer.step()

CPU times: user 771 ms, sys: 630 ms, total: 1.4 s
Wall time: 3.08 s


In [ ]:
import os

In [ ]:
os.system("nvidia-smi")

Wed May 10 19:26:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:3B:00.0 Off |                    0 |
|  0%   38C    P8    33W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          On   | 00000000:5E:00.0 Off |                    0 |
|  0%   

0

In [ ]:
out.shape

torch.Size([4, 3, 128, 128, 128])

In [ ]:
count_learnable_parameters(model.vit)

122871594

In [ ]:
count_learnable_parameters(model.vit.embedders)

41945605

In [ ]:
model.vit.embedders

In [ ]:
count_learnable_parameters(SWINUNETR)

In [ ]:
count_learnable_parameters(UNet)

In [ ]:
import torch
import torch.nn as nn

batch_size = 16
in_channels = 3
x_dim, y_dim, z_dim = 128, 128, 128
patch_size = 8
embedding_dim = 512

class PatchEmbedding(nn.Module):
    def __init__(self, patch_size, in_channels, embedding_dim):
        super().__init__()
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embedding_dim = embedding_dim
        self.projection = nn.Linear(in_channels * patch_size**3, embedding_dim)
        
    def forward(self, x):
        # x shape: (batch, channels, x, y, z)
        batch_size, channels, x_dim, y_dim, z_dim = x.shape

        # Create non-overlapping patches
        patches = x.unfold(2, self.patch_size, self.patch_size)
        patches = patches.unfold(3, self.patch_size, self.patch_size)
        patches = patches.unfold(4, self.patch_size, self.patch_size)

        # Flatten patches
        patches_flat = patches.contiguous().view(batch_size, -1, channels * self.patch_size**3)

        # Apply linear embeddings
        embeddings = self.projection(patches_flat)

        return patches_flat, patches.shape

import torch
import torch.nn as nn

class InversePatchEmbedding(nn.Module):
    def __init__(self, patch_size, in_channels, embed_dim):
        super().__init__()
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim
        self.projection = nn.Linear(embed_dim, in_channels * patch_size**3)

    def forward(self, embeddings, patches_shape):
        # Apply inverse linear embeddings
        # patches_flat = self.projection(embeddings)

        # Reshape patches
        print(patches_shape)
        batch_size, _, channels, x_unfold, y_unfold, z_unfold = patches_shape
        patches = embeddings.view(batch_size, -1, channels, self.patch_size, self.patch_size, self.patch_size)

        # Combine non-overlapping patches
        x_dim = x_unfold * self.patch_size
        y_dim = y_unfold * self.patch_size
        z_dim = z_unfold * self.patch_size

        patches = patches.view(batch_size, x_unfold, y_unfold, z_unfold, channels, self.patch_size, self.patch_size, self.patch_size)
        x = patches.permute(0, 4, 1, 5, 2, 6, 3, 7).reshape(batch_size, channels, x_dim, y_dim, z_dim)

        return x

# Example usage
input_tensor = torch.randn(batch_size, in_channels, x_dim, y_dim, z_dim)
patch_embedding = PatchEmbedding(patch_size, in_channels, embedding_dim)
output, patch_shape = patch_embedding(input_tensor)
print(output.shape)  # Output: (16, 512, 64)

inverse_patch_embedding = InversePatchEmbedding(patch_size, in_channels, embedding_dim)
reconstructed_input = inverse_patch_embedding(output, patch_shape)
print(reconstructed_input.shape)  # Output: (16, 3, 32, 32, 32)

In [ ]:
input_tensor == reconstructed_input

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, patch_size, in_channels, embedding_dim):
        super().__init__()
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embedding_dim = embedding_dim
        self.projection = nn.Linear(in_channels * patch_size**3, embedding_dim)

    def forward(self, x):
        batch_size, channels, x_dim, y_dim, z_dim = x.shape

        patches_x = x.unfold(2, self.patch_size, self.patch_size)
        patches_y = patches_x.unfold(3, self.patch_size, self.patch_size)
        patches_z = patches_y.unfold(4, self.patch_size, self.patch_size)

        patches = patches_z.permute(0, 2, 3, 4, 1, 5, 6, 7).contiguous()
        patches = patches.view(batch_size, -1, self.patch_size**3 * channels)

        embeddings = self.projection(patches)

        return embeddings

In [ ]:
class ReversePatchEmbedding(nn.Module):
    def __init__(self, patch_size, in_channels, embedding_dim):
        super().__init__()
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embedding_dim = embedding_dim
        self.projection = nn.Linear(embedding_dim, in_channels * patch_size**3)

    def forward(self, embeddings, x_shape):
        batch_size, _, x_dim, y_dim, z_dim = x_shape

        patches = self.projection(embeddings)

        patches = patches.view(batch_size, -1, self.in_channels, self.patch_size, self.patch_size, self.patch_size)

        num_patches_x = x_dim // self.patch_size
        num_patches_y = y_dim // self.patch_size
        num_patches_z = z_dim // self.patch_size

        patches = patches.view(batch_size, num_patches_x, num_patches_y, num_patches_z, self.in_channels, self.patch_size, self.patch_size, self.patch_size)

        x = patches.permute(0, 4, 1, 5, 2, 6, 3, 7).contiguous()
        x = x.view(batch_size, self.in_channels, x_dim, y_dim, z_dim)

        return x

In [ ]:
example = torch.rand(4, 32, 128, 128, 128)
p = PatchEmbedding(8, 32,  512)
r = ReversePatchEmbedding(8, 32, 512)

In [ ]:
out = r(p(example), example.shape)

In [ ]:
torch.allclose(example, out, atol=1e-6)